In [1]:
!pip install pymysql # mysql database connection library
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
!python3 -m pip install --upgrade "git+https://github.com/nficano/pytube.git"   # pytube-12.0.0 설치
! pip install pydub

import os
import time                     # 딜레이 주기 위함
import datetime
import numpy as np
import pandas as pd             # Querying results to make dataframe(to divide datas)
import pymysql.cursors          # For Request Query String
from google.colab import drive

import pytube
from pytube import Search
from pydub import AudioSegment

drive.mount('/content/drive', force_remount=True)

Traceback (most recent call last):
  File "/usr/local/bin/jupyter-notebook", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/jupyter_core/application.py", line 264, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 88, in inner
    return method(app, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/notebook/notebookapp.py", line 1507, in initialize
    self.init_webapp()
  File "/usr/local/lib/python3.7/dist-packages/notebook/notebookapp.py", line 1297, in init_webapp
    self.http_server.listen(port, self.ip)
  File "/usr/local/lib/python3.7/dist-packages/tornado/tcpserver.py", line 143, in listen
    sockets = bind_sockets(port, address=address)
  File "/usr/local/lib/pytho

In [9]:
#================================================장르별 검색 루프===============================================================================
# (genre = DB의 장르 검색 및 폴더 생성 기준, 유튜브 비디오 몇번 째, 기본 저장 경로, db 조회결과 몇 개까지, db가 아닌 누락으로 재 다운시 'unfounded_lists' 입력)
def get_genre(genre, video_index, path, limit, result=None):     
        conn = pymysql.connect(host='csrd.cksngv0eixsu.ap-northeast-2.rds.amazonaws.com', port=3306, user='admin', password='qwerty1!', \
        db='before-pp', charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor)        # mysql connect inform
        sql = f"SELECT c.sg_name, c.sg_artists FROM bf_pl a join bf_tg b on a.pl_id = b.pl_id join bf_sg c on c.pl_id = b.pl_id WHERE pl_intro LIKE '{genre}%'"
        cur = conn.cursor() # open to query
        cur.execute(sql)    # excute query
        result = cur.fetchall() # merging result
        result_ = result
        conn.close()    # DB disconnect
        df = pd.DataFrame(result_)   # 전체 음악리스트 확인(playlist_id 기준이기 때문에 중복값 들어있음)
        df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)  # 중복 처리
        df = df[:int(float(limit))]
        ext_lists = df.values.tolist() # pytube에서 검색어로 사용하기 위한 리스트 변환

#================================================음원 검색===============================================================================
        print(f"{genre} 장르로 유튜브 검색 시작")
        audio_lists = []    # 검색결과가 나온 비디오 넣는 리스트
        forbidden_lists = []    # 검색결과 없는 비디오 리스트 - 추후 DB 적용하기 위하여
        for ext in ext_lists:
            try:
                StrClean = "-".join(ext)            # 노래 제목과 가수를 "-"를 구분자로 문자열 합치기
                yt = Search(StrClean).results[video_index]    # Search 메소드를 사용하여 검색하고, 그 결과의 1번째 결과 사용(빈도 상, 1 ~ 2번째 영상에 가장 관련있는 영상 표시)
            except:
                print("=================== Error Occured :  " + StrClean + " ===================")  # 에러 로그
                forbidden_lists.append(StrClean)    # 검색결과 없는 비디오 리스트 삽입
            audio_lists.append({StrClean : yt})     # 성공적으로 검색되는 비디오의 Youtube 객체 리스트 삽입
        # 추후 파일명으로 사용하여 라벨링 및 DB 적용을 수월하게 하기 위하여 딕셔너리 형태로 전달
        print(f"{genre} 장르로 유튜브 검색 완료")
#================================================음원 다운로드===============================================================================
        
        unfounded_lists = []    # 검색된 영상 중 필터링된, 관련성 떨어지는 목록용 리스트
        fail_cnt = 0            # 다운로드 실패 횟수 초기화
        for fail_cnt ,audio in enumerate(audio_lists):
            try:
                keys = str(list(audio.items())[0][0])   # 딕셔너리에서 파일 이름으로 쓸 string 언패킹
                link = audio.get(keys)                     # 딕셔너리에서 파일이름이 key값이므로, values 값인  (pytube.__main__.YouTube) 자료형 언패킹
                audio_streams = link.streams.filter(only_audio=True)    # pytube에서 지원하는 다양한 stream 중 Audio 속성을 가진 stream만
                # print(audio_streams)  # audio streams 처리 전체 정보

                flt1a = (int(link.length)) > 130    # 영상 길이 2분 10초 초과
                if genre == "클래식" or "classic":
                    flt1b = (int(link.length)) < 1800    # 그래도 클래식이니까 영상 길이 30분 미만
                else:
                    flt1b = (int(link.length)) < 600    # 클래식 제외 10분 미만
                flt1 = flt1a and flt1b              # 위의 두 조건 동시에 만족하는 필터
                flt2 = ("official" in str(link.title).lower() or "audio" in str(link.title).lower() or "lylic" in str(link.title).lower())  
                    # "official", "audio", "lylic" 중 제목에 하나라도 들어가는 영상 제목 통과

                # 내용 체크
                print(f'영상제목: {link.title},  영상 조회수: {link.views},  영상 길이: {link.length} sec. [{str(link.length // 60).zfill(2)}:{str(link.length % 60).zfill(2)}]')
                print(f'영상 URL: {link.watch_url}')
                print(f"# 1번 체크포인트 - 음원 길이(초):  {int(link.length)}, 타입은? : {type(int(link.length))}")
                print(f"# 2번 체크포인트 - 음원 제목(소문자):  {str(link.title).lower()}") 
                print(f"# 3번 체크포인트 - 2분 이상 20분 이하 길이:  {flt1}  ,  음원에 자주 붙은 문자열 포함 여부: {flt2}")


                # 저장 폴더 만들기
                genre_dir = f"{genre}"
                dir =  f"{path}/{genre_dir}"
                if not os.path.exists(dir):
                    os.mkdir(dir)

                                                                                                # filter()를 활용하여 파일 확장자 타입 등을 포함한 streams 처리
                if flt1 or flt2 == True:                                                        # 위의 1, 2 조건 중 한가지라도 만족한다면 다운로드
                    audio_streams = link.streams.filter(file_extension='mp4').get_by_itag(140)  # 여러개의 다운로드 스트림 선택해서 링크 하나 가져오기
                    title = keys                                                                # 영상 제목 파일명: audio_streams.title 또는 link.title 사용 
                    audio_streams.download(filename= title + '.mp3', output_path= dir)          # 설정한 파일명, 경로에 다운로드
                    time.sleep(2)                                                               # 2초 휴지
                    print(f"#@#@#@#@#@#@#@ {keys} 다운로드 성공!")                              # 성공 로그
                    print("============================================== 구분선 ==============================================")
                else:
                    print(f"^^^^^^^^^^^ Related Video Not Found : {keys}")                      # 실패 로그
                    unfounded_lists.append(keys)                                                # 실패 목록 리스트에 저장
                    print("============================================== 구분선 ==============================================")
                    return unfounded_lists
            except:
                    print(f"no error keep going, {fail_cnt}개째 다운로드 실패")       

In [10]:
# (genre = 컨버팅할 최종 경로 및 장르, path = 기본 컨버팅 차상위 폴더 경로)
### 다운로드 받은 음원 파일을 mp4 to mp3 to wav and 재생구간을 st * 10초 ~ ed * 10초 구간만 사용 // st와 ed가 없을경우 
def genre_converter(genre, path_, st=None, ed=None):
    print(f"{genre} 폴더 음원 컨버팅 시작")                     # 시작할 폴더명 로그
    genre_dir = f"{genre}/"                                     # 변수 재정의
    dir  = f"{path_}/{genre}"                                   # 컨버팅 할 폴더
    try:                                                      # 빈 폴더 및 에러 방지용
        def song_names(dir):                                    # 폴더 내의 파일 읽어오는 함수
            files = list(os.listdir(dir))
            return files    # filename
        filename = song_names(dir)                              # song_names 실행하여 각각의 파일들을 filename에 저장

        def load_song(filename):                                # filename에 있는 파일들을 저장 경로와 음악 이름으로 분리하여 변수 저장하는 함수
            song = f'{filename}'
            path = f'{dir}/{filename}'
            return song, path

        songs = []
        paths = []

        for file in filename:                                   # load_song 함수를 반복 사용하여 filename를 저장경로와 파일이름으로 분리하여 리스트에 저장하는 부분
            song, path = load_song(file)
            songs.append(f"{song}")
            paths.append(f"{path}")

        for file in paths:                                      # 각 경로 별로 파일들을 불러와 pydub으로 확장자를 변환하고, converted 폴더를 생성하여 저장. 추가로 재생구간을 잘라서 저장할 수 있음
            filename, file_extension = os.path.splitext(file)   # os.path를 사용하여 파일이름과 확장자를 분리
            if file_extension == ".mp3":                        # 확장자가 mp3일 경우 파일 형식을 wav로 바꾸어 Decoding(wav파일보다 mp3파일이 압축률이 높음)
                dst = file.replace(".mp3","")+".wav"                                                             
                audSeg = AudioSegment.from_file(file, format="mp4") # convert wav from mp3 
                audSeg.export(dst, format="wav")                    # wav 파일로 변환 
                song = AudioSegment.from_wav(dst)

                sec = 10000                                         # 10초 변수
                if st and ed == None:       # 음원 파일의 원하는 구간만 사용하기 위한 조건문
                    pass                                            # 구간추출 설정 안할 경우 해당 프로세스 무시
                elif st == None:                        # 종료 10ed초만 설정 시 처음부터 10ed초까지로 구간자름                        
                    end = int(float(ed)) * sec
                    block = song[:end] 
                elif ed == None:                        # 시작 10st초만 설정 시 10st초부터 끝까지로 구간자름             
                    start = int(float(st)) * sec
                    block = song[start:] 
                else:                                               # 시작과 종료 모두 설정시 10st ~ 10ed 구간으로 자름            
                    start = int(float(st)) * sec
                    end = int(float(ed)) * sec
                    block = song[start:end]                         # start초 ~ end초 구간만 사용하기 위한 변수(하이라이트 추출은 80초부터 110초까지)
                
                down_dir = path_                                                                                     # 경로 변수 재정의
                inner_dir = f"converted/"                                                                            # 컨버팅 된 파일 저장할 내부 폴더 이름 정의
                dir_full = f"{down_dir}/{genre_dir}{inner_dir}"                                                      # 변환된 파일 저장 경로                                                
                if not os.path.exists(dir_full):                                                                     # 폴더 없을 경우만 생성
                    os.mkdir(dir_full)
                name = f'{file.replace(f"{down_dir}/","").replace(f"{genre_dir}","").replace(".mp3","")}_{(10*ed) - (10*st)}.wav'    # 기존 파일의 확장자명을 알맞게 바꾸고 구분할 수 있는 문자("_30")
                export_path = dir_full + name   
                block.export(export_path, format='wav', parameters=["-q:a", "10", "-ac", "1"])                       # 설정한 파라미터 값과 설정한 구간으로 잘라서 변환
                print(f"Converting Successful // converted filename: {export_path}")                                 # 컨버팅 성공 로그
            else:
                print(f"{file} 파일이 컨버팅 실패하였습니다.")                                                       # 컨버팅 실패 로그                                
    except:
        print(f"There is no '{genre}' folder.")                                                                      # 폴더 없을 경우 에러메세지 로그

In [11]:
path = f"/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/genre_test"
genre_lists = ['락', '클래식', '메탈', '디스코', '블루스', '아이돌', '트로트', '힙합', '재즈', '팝', '알앤비']
# genre_lists = ['락',  '메탈', '블루스',  '힙합', '재즈', '팝', '알앤비']

for each_genre in genre_lists:
    get_genre(each_genre, 0, path, 3)            # 장르 // 유튜브 비디오 검색결과 몇번째 인덱스 // 기본 경로 // DB 검색 레코드 개수
    genre_converter(each_genre, path, 8, 11)     # 장르 // 기본 경로 // 음원 자르기 시작 구간 // 음원 자르기 끝 구간

락 장르로 유튜브 검색 시작
락 장르로 유튜브 검색 완료
영상제목: Megadeth - Conquer Or Die,  영상 조회수: 7271615,  영상 길이: 215 sec. [03:35]
영상 URL: https://youtube.com/watch?v=dQ_-tUKT-nY
# 1번 체크포인트 - 음원 길이(초):  215, 타입은? : <class 'int'>
# 2번 체크포인트 - 음원 제목(소문자):  megadeth - conquer or die
# 3번 체크포인트 - 2분 이상 20분 이하 길이:  True  ,  음원에 자주 붙은 문자열 포함 여부: False
#@#@#@#@#@#@#@ Conquer Or Die-Megadeth 다운로드 성공!
============================================== 구분선 ==============================================
영상제목: Nirvana - Smells Like Teen Spirit (Official Music Video),  영상 조회수: 1447405357,  영상 길이: 278 sec. [04:38]
영상 URL: https://youtube.com/watch?v=hTWKbfoikeg
# 1번 체크포인트 - 음원 길이(초):  278, 타입은? : <class 'int'>
# 2번 체크포인트 - 음원 제목(소문자):  nirvana - smells like teen spirit (official music video)
# 3번 체크포인트 - 2분 이상 20분 이하 길이:  True  ,  음원에 자주 붙은 문자열 포함 여부: True
#@#@#@#@#@#@#@ Smells Like Teen Spirit-Nirvana 다운로드 성공!
============================================== 구분선 ==============================================
영상제목: Rage Against

Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Don't Start Now-Dua Lipa
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.
Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: New Rules-Dua Lipa
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.


디스코 장르로 유튜브 검색 완료
영상제목: Dua Lipa - Levitating Featuring DaBaby (Official Music Video),  영상 조회수: 491010340,  영상 길이: 230 sec. [03:50]
영상 URL: https://youtube.com/watch?v=TUVcZfQe-Kw
# 1번 체크포인트 - 음원 길이(초):  230, 타입은? : <class 'int'>
# 2번 체크포인트 - 음원 제목(소문자):  dua lipa - levitating featuring dababy (official music video)
# 3번 체크포인트 - 2분 이상 20분 이하 길이:  True  ,  음원에 자주 붙은 문자열 포함 여부: True
#@#@#@#@#@#@#@ Levitating (feat. DaBaby)-Dua Lipa 다운로드 성공!
============================================== 구분선 ==============================================
영상제목: Dua Lipa - Don't Start Now (Official Music Video),  영상 조회수: 580610058,  영상 길이: 181 sec. [03:01]
영상 URL: https://youtube.com/watch?v=oygrmJFKYZY
# 1번 체크포인트 - 음원 길이(초):  181, 타입은? : <class 'int'>
# 2번 체크포인트 - 음원 제목(소문자):  dua lipa - don't start now (official music video)
# 3번 체크포인트 - 2분 이상 20분 이하 길이:  True  ,  음원에 자주 붙은 문자열 포함 여부: True
#@#@#@#@#@#@#@ Don't Start Now-Dua Lipa 다운로드 성공!
============================================== 구분선 =================